# Limpieza

In [1]:
# Importamos las librerias necesarias
import pandas as pd
from pandas.api.types import CategoricalDtype

In [2]:
# Cargamos los datos
data = pd.read_csv('source/data/historic_demand_2009_2022_noNaN.csv')
copy = data     # Creamos una copia de los datos y realizamos las modificaciones en ese dataframe
copy.head()     # Primera impresión

,Unnamed: 0,settlement_date,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow
0,0,2009-01-01 00:30:00,1,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0
1,1,2009-01-01 01:00:00,2,38047,38964,34072,53,1403,0,0,0,157,2002,0,0,-160,0,0
2,2,2009-01-01 01:30:00,3,37380,38651,33615,53,1403,0,0,0,511,2002,0,0,-160,0,0
3,3,2009-01-01 02:00:00,4,36426,37775,32526,50,1403,0,0,0,589,1772,0,0,-160,0,0
4,4,2009-01-01 02:30:00,5,35687,37298,31877,50,1403,0,0,0,851,1753,0,0,-160,0,0


### Modificaciones

In [3]:
# Eliminamos la variable Unnamed, solo representa el indíce incremental
copy.drop('Unnamed: 0', axis=1, inplace=True)
copy.dropna(inplace=True)

# Seteamos el nuevo indíce mediante settlement_date y le ponemos su correspondiente tipo de dato (datetime)
copy = copy.set_index('settlement_date')
copy.index = pd.to_datetime(copy.index)

#### Creamos nuevas features

In [4]:
# Creamos nuevas features desglozando la variable settlement_date
copy['day'] = copy.index.day
copy['month'] = copy.index.month
copy['year'] = copy.index.year
copy['hour'] = copy.index.hour
copy['minute'] = copy.index.minute
copy['quarter'] = copy.index.quarter

copy['date_offset'] = (copy.index.month*100 + copy.index.day - 320)%1300
copy['season'] = pd.cut(copy['date_offset'], [0, 300, 602, 900, 1300], labels=['Spring', 'Summer', 'Fall', 'Winter'])

cat_type = CategoricalDtype(categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], ordered=True)

copy['weekday'] = copy.index.day_name()
copy['weekday'] = copy['weekday'].astype(cat_type)

#### Modificamos los valores de la variable tsd

In [5]:
# Demanda (en MW) de los sistemas de transmisión  
copy['tsd'] = copy['tsd'] - copy['nd']

### Verificamos que se hayan realizado los cambios

In [6]:
copy.day.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 29, 24, 25, 26, 27, 28, 30, 31],
      dtype=int64)

In [7]:
copy.hour.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23], dtype=int64)

In [8]:
copy.month.unique()

array([ 1,  2,  8,  5,  3,  6,  4,  7,  9, 10, 12, 11], dtype=int64)

In [9]:
copy.year.unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022], dtype=int64)

In [10]:
copy.quarter.unique()

array([1, 3, 2, 4], dtype=int64)

In [11]:
copy.tsd.unique()

array([ 794,  917, 1271, ..., 5051, 4934, 5046], dtype=int64)

In [12]:
# Elegimos una muestra de 5 valores para visualizar los cambios en el dataframe
copy.sample(5)

,settlement_period,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,...,nemo_flow,day,month,year,hour,minute,quarter,date_offset,season,weekday
settlement_date,,,,,,,,,,,,,,,,,,,,,
2014-07-30 04:30:00,9,21133,2203,19415,810,3344,0,5057,0,1470,...,0,30,7,2014,4,30,3,410,Summer,Wednesday
2021-06-01 19:30:00,39,29758,903,27053,1327,6527,1420,13080,0,7,...,1013,1,6,2021,19,30,2,281,Spring,Tuesday
2019-06-12 01:00:00,2,20178,896,19181,2325,6137,0,13080,0,396,...,668,12,6,2019,1,0,2,292,Spring,Wednesday
2011-07-02 02:30:00,5,23121,1931,21032,16,1828,0,454,0,1193,...,0,2,7,2011,2,30,3,382,Summer,Saturday
2014-01-14 00:00:00,48,31491,2160,28380,1020,2524,0,3393,0,934,...,0,14,1,2014,0,0,1,1094,Winter,Tuesday


Vemos que se modificaron los valores de la demanda de los sistemas de transmisión y se agregaron las nuevas features.

### Creamos un nuevo dataframe con los datos modificados y lo guardamos para el análisis

In [13]:
df = pd.DataFrame(
    {
    'Hour': copy.hour.values,
    'Minute': copy.minute.values,
    'Day': copy.day.values,
    'Weekday': copy.weekday.values,
    'Month': copy.month.values,
    'Year': copy.year.values,
    'Quarter': copy.quarter.values,
    'Season': copy.season.values,
    'ND': copy.nd.values,
    'TDS': copy.tsd.values,
    'England_Wales_Demand': copy.england_wales_demand.values,
    'Embedded_Wind_Generation': copy.embedded_wind_generation.values,
    'Embedded_Wind_Capacity': copy.embedded_wind_capacity.values,
    'Embedded_Solar_Generation': copy.embedded_solar_generation.values, 
    'Embedded_Solar_Capacity': copy.embedded_solar_capacity.values,
    'Non_BMSTOR': copy.non_bm_stor.values,
    'Pump_Storage_Pumping': copy.pump_storage_pumping.values,
    'IFA': copy.ifa_flow.values,
    'IFA2': copy.ifa2_flow.values,
    'Britned': copy.britned_flow.values,
    'Moyle': copy.moyle_flow.values,
    'East_West': copy.east_west_flow.values,
    'Nemo': copy.nemo_flow.values
    }
)
df

,Hour,Minute,Day,Weekday,Month,Year,Quarter,Season,ND,TDS,...,Embedded_Solar_Generation,Embedded_Solar_Capacity,Non_BMSTOR,Pump_Storage_Pumping,IFA,IFA2,Britned,Moyle,East_West,Nemo
0,0,30,1,Thursday,1,2009,1,Winter,37910,794,...,0,0,0,33,2002,0,0,-161,0,0
1,1,0,1,Thursday,1,2009,1,Winter,38047,917,...,0,0,0,157,2002,0,0,-160,0,0
2,1,30,1,Thursday,1,2009,1,Winter,37380,1271,...,0,0,0,511,2002,0,0,-160,0,0
3,2,0,1,Thursday,1,2009,1,Winter,36426,1349,...,0,0,0,589,1772,0,0,-160,0,0
4,2,30,1,Thursday,1,2009,1,Winter,35687,1611,...,0,0,0,851,1753,0,0,-160,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242751,22,0,6,Sunday,11,2022,4,Fall,25340,1207,...,0,13080,0,408,228,757,1000,293,19,999
242752,22,30,6,Sunday,11,2022,4,Fall,23826,2347,...,0,13080,0,87,-811,-301,1001,378,132,999
242753,23,0,6,Sunday,11,2022,4,Fall,22620,2941,...,0,13080,0,464,-894,-400,981,400,45,988
242754,23,30,6,Sunday,11,2022,4,Fall,21068,4217,...,0,13080,0,734,-923,-963,158,400,70,307


In [ ]:
# Exportamos el dataframe modificado para su posterior análisis
df.to_csv('source/data/clean_data.csv', index=False)